# Having fun with DeepBind

In [ ]:
from depiction.models.examples.deepbind.deepbind import DeepBind
from depiction.core import Task, DataType
from depiction.interpreters.u_wash.u_washer import UWasher
from depiction.models.examples.deepbind.deepbind import create_DNA_language
from ipywidgets import interact

## Setup task

In [ ]:
class_names = ['NOT BINDING', 'BINDING']
tf_factor_id = 'D00328.003' # CTCF
classifier = DeepBind(tf_factor_id = tf_factor_id)

# Instantiate the interpreters

In [ ]:
# Create a LIME text interpreter
interpreter = 'lime'
lime_explanation_configs = {
    'labels': (1,),
}
interpreter_params = {
    'class_names': class_names,
    'split_expression': list,
    'bow': False,
    'char_level': True
}

lime_explainer = UWasher(interpreter, classifier, **interpreter_params)

# Create Anchor text intepreter
interpreter = 'anchors'
anchors_explanation_configs = {
    'use_proba': False,
    'batch_size': 100
}
interpreter_params = {
    'class_names': class_names,
    'nlp': create_DNA_language(),
    'unk_token': 'N',
    'sep_token': '',
    'use_unk_distribution': True
}

anchor_explainer = UWasher(interpreter, classifier, **interpreter_params)

### Wrapper for the interactive widget

In [ ]:
class InteractiveWrapper:
    def __init__(self, classifier):
        self.classifier = classifier
        
    def callback(self, sequence):
        # LIME
        self.classifier.use_labels = False
        lime_explainer.interpret(sequence, explanation_configs=lime_explanation_configs)
        # Anchors 
        self.classifier.use_labels = True
        anchor_explainer.interpret(sequence, explanation_configs=anchors_explanation_configs)

# Let's interpret

In [ ]:
wrapper = InteractiveWrapper(classifier)

interact(
    wrapper.callback,
    sequence=[
        'AGGCTAGCTAGGGGCGCCC', 'AGGCTAGCTAGGGGCGCTT', 'AGGGTAGCTAGGGGCGCTT',
        'AGGGTAGCTGGGGGCGCTT', 'AGGCTAGGTGGGGGCGCTT', 'AGGCTCGGTGGGGGCGCTT',
        'AGGCTCGGTAGGGGGCGATT'
    ]
)

CTCF binding motif
![CTCF binding motif](https://media.springernature.com/full/springer-static/image/art%3A10.1186%2Fgb-2009-10-11-r131/MediaObjects/13059_2009_Article_2281_Fig2_HTML.jpg?as=webp)
from Essien, Kobby, et al. "CTCF binding site classes exhibit distinct evolutionary, genomic, epigenomic and transcriptomic features." Genome biology 10.11 (2009): R131.

In [ ]:
tf_factor_id = 'D00761.001' # FOXA1
classifier.tf_factor_id = tf_factor_id
wrapper = InteractiveWrapper(classifier)

interact(wrapper.callback, sequence='TGTGTGTGTG')

FOXA1 binding motif
![FOXA1 binding motif](https://ismara.unibas.ch/supp/dataset1_IBM_v2/ismara_report/logos/FOXA1.png)
from https://ismara.unibas.ch/supp/dataset1_IBM_v2/ismara_report/pages/FOXA1.html